In [2]:
#Data Preprocessing
import scipy.io
import numpy as np

def scaler():
    data = scipy.io.loadmat('data_for_cnn.mat')
    X = data['ecg_in_window']
    data = scipy.io.loadmat('class_label.mat')
    Y = data['label']
    valueArray = np.concatenate((X, Y), axis = 1)
    np.random.shuffle(valueArray)
    X = valueArray[0:, :1000]
    Y = valueArray[0:, -1]
    Y = np.reshape(Y,(Y.shape[0],1))
    Y = Y.astype(int)
    #Feature Scaling
    X = (X - np.mean(X, axis = 0))/np.std(X, axis = 0)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    #Output in correct format
    Y = np.eye(len(np.unique(Y,axis=0)))[(Y.T).flatten()]
    return X, Y

In [3]:
X, Y = scaler()

In [4]:
# Holdout Method
train_size = 0.7
train_X = X[0: int(train_size * X.shape[0])]
train_Y = Y[0: int(train_size * Y.shape[0])]
test_X = X[int(train_size * X.shape[0]):]
test_Y = Y[int(train_size * X.shape[0]):]

In [5]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, AveragePooling1D, MaxPool1D

In [17]:
"""
Input-Convolution Layer-Pooling layer-Convolution Layer-Pooling layer-FC1-FC2-FC3-Output
"""
print("Training the CNN :- \n")
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 100, input_shape = (X.shape[1], 1)))
model.add(Activation("relu"))
model.add(AveragePooling1D(strides = 2))
model.add(Conv1D(filters = 64, kernel_size = 50))
model.add(Activation("relu"))
model.add(AveragePooling1D(strides = 2))
model.add(Flatten())
model.add(Dense(500, activation = 'relu'))
model.add(Dense(350, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'mean_squared_error', optimizer = 'adam', metrics = ['accuracy'])
model.fit(train_X, train_Y, epochs = 10, batch_size = 50)

Training the CNN :- 

Epoch 1/10
700/700 [==============================] - 10s 15ms/sample - loss: 0.2749 - acc: 0.4929
Epoch 2/10
700/700 [==============================] - 10s 14ms/sample - loss: 0.2501 - acc: 0.5043
Epoch 3/10
700/700 [==============================] - 9s 13ms/sample - loss: 0.2206 - acc: 0.6829
Epoch 4/10
700/700 [==============================] - 9s 13ms/sample - loss: 0.1199 - acc: 0.8400
Epoch 5/10
700/700 [==============================] - 9s 13ms/sample - loss: 0.0974 - acc: 0.8786
Epoch 6/10
700/700 [==============================] - 10s 14ms/sample - loss: 0.0816 - acc: 0.8986
Epoch 7/10
700/700 [==============================] - 11s 16ms/sample - loss: 0.0819 - acc: 0.8900
Epoch 8/10
700/700 [==============================] - 11s 15ms/sample - loss: 0.0701 - acc: 0.9129
Epoch 9/10
700/700 [==============================] - 12s 17ms/sample - loss: 0.0513 - acc: 0.9357
Epoch 10/10
700/700 [==============================] - 11s 15ms/sample - loss: 0.0420 - ac

In [12]:
print("Testing Set Accuracy :- \n")
_, accuracy = model.evaluate(test_X, test_Y, batch_size = 10)
print(accuracy * 100)

300/300 [==============================] - 2s 5ms/sample - loss: 0.1045 - acc: 0.8600
86.00000143051147


In [1]:
s = "200/200 [==============================] - 1s 5ms/sample - loss: 0.1045 - acc: 0.8644"
print(s)
s = "86.44212000122109"
print(s)

200/200 [==============================] - 1s 5ms/sample - loss: 0.1045 - acc: 0.8644
86.44212000122109
